<a href="https://colab.research.google.com/github/vectara/example-notebooks/blob/main/notebooks/hallucination_mitigation/vhc-llamaindex-integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VHC (Vectara Hallucination Corrector) with LLamaIndex Integration

This notebook demonstrates how to integrate Vectara's HHEM (Hughes Hallucination Evaluation Model) and VHC (Vectara Hallucination Corrector) with standard LlamaIndex RAG workflow.

## Installation and Setup

In [8]:
!pip install --quiet llama-index llama-index-llms-openai llama-index-embeddings-openai llama-index-vector-stores-chroma requests python-dotenv chromadb termcolor

## Environment Setup

Set up your environment variables. You'll need:
- `VECTARA_API_KEY`: Your Vectara API key (for HHEM and VHC)
- `OPENAI_API_KEY`: Your OpenAI API key (for LlamaIndex RAG)

In [2]:
import os
import json
import requests
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv
from termcolor import colored

# Load environment variables
load_dotenv()

# Set your API keys here or in your environment
os.environ["VECTARA_API_KEY"] = os.getenv("VECTARA_API_KEY", "<YOUR_VECTARA_API_KEY>")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "<YOUR_OPENAI_API_KEY>")

# Verify API keys are set
if not os.getenv('VECTARA_API_KEY') or os.getenv('VECTARA_API_KEY') == '<YOUR_VECTARA_API_KEY>':
    raise EnvironmentError("VECTARA_API_KEY environment variable is not set.")

if not os.getenv('OPENAI_API_KEY') or os.getenv('OPENAI_API_KEY') == '<YOUR_OPENAI_API_KEY>':
    raise EnvironmentError("OPENAI_API_KEY environment variable is not set.")

print("Environment variables configured successfully")

Environment variables configured successfully


## Vectara HHEM and VHC Client

Create clients for interacting with Vectara's HHEM and VHC endpoints:

In [3]:
class VectaraClient:
    """Client for interacting with Vectara HHEM and VHC endpoints"""
    
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.session = requests.Session()
        self.base_url = "https://api.vectara.io"
    
    def evaluate_factual_consistency(self, query: str, response: str, documents: List[str]) -> Dict[str, Any]:
        """Evaluate factual consistency using HHEM"""
        
        payload = {
            "generated_text": response,
            "source_texts": documents
        }
        
        headers = {
            "Content-Type": "application/json",
            "Accept": "application/json",
            "x-api-key": self.api_key
        }
        
        try:
            response = self.session.post(
                f"{self.base_url}/v2/evaluate_factual_consistency",
                json=payload,
                headers=headers,
                timeout=30
            )
            response.raise_for_status()
            return response.json()
            
        except requests.exceptions.RequestException as e:
            raise RuntimeError(f"HHEM API request failed: {e}")
        except json.JSONDecodeError as e:
            raise RuntimeError(f"Failed to parse HHEM response: {e}")
    
    def correct_hallucinations(
        self, 
        query: str, 
        generated_text: str, 
        documents: List[str],
        model_name: str = "vhc-large-1.0"
    ) -> Dict[str, Any]:
        """Correct hallucinations using VHC"""
        
        payload = {
            "generated_text": generated_text,
            "query": query,
            "documents": [{"text": doc} for doc in documents],
            "model_name": model_name
        }
        
        headers = {
            "Content-Type": "application/json",
            "Accept": "application/json",
            "x-api-key": self.api_key
        }
        
        try:
            response = self.session.post(
                f"{self.base_url}/v2/hallucination_correctors/correct_hallucinations",
                json=payload,
                headers=headers,
                timeout=60
            )
            response.raise_for_status()
            
            data = response.json()
            corrected_text = data.get("corrected_text", "")
            corrections = data.get("corrections", [])
            
            return {
                "corrected_text": corrected_text,
                "corrections": corrections,
                "original_text": generated_text
            }
            
        except requests.exceptions.RequestException as e:
            raise RuntimeError(f"VHC API request failed: {e}")
        except json.JSONDecodeError as e:
            raise RuntimeError(f"Failed to parse VHC response: {e}")

# Initialize Vectara client
vectara_client = VectaraClient(os.getenv("VECTARA_API_KEY"))

## Setup: LlamaIndex RAG Engines

Let's create two different LlamaIndex RAG engines.
The first one will respond with "I don't know" if it cannot respond based on the source text
The second one is instructed to use its internal knowledge, which can result in a hallucination relative to the RAG information.

In [4]:
from llama_index.core import VectorStoreIndex, Document, Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.prompts import PromptTemplate
from llama_index.core.retrievers import VectorIndexRetriever
import chromadb

# Initialize LLM and embeddings
Settings.llm = OpenAI(model="gpt-4o", temperature=0.0)
Settings.embed_model = OpenAIEmbedding()

# Create knowledge base with more detailed but still limited information
sample_docs = [
    "The Eiffel Tower is a wrought-iron lattice tower located in Paris, France. It was designed by Alexandre Gustave Eiffel.",
    "Leonardo da Vinci was an Italian Renaissance polymath who painted the Mona Lisa. He lived from 1452 to 1519.",
    "William Shakespeare was an English playwright and poet who wrote Romeo and Juliet. He is considered the greatest writer in the English language.",
    "The Great Wall of China is an ancient series of walls and fortifications built to protect Chinese states from invasions.",
    "Albert Einstein was a German-born theoretical physicist who developed the theory of relativity. He won the Nobel Prize in Physics in 1921.",
    "The Statue of Liberty is a neoclassical sculpture located on Liberty Island in New York Harbor.",
    "Mount Everest is the Earth's highest mountain located in the Himalayas on the border between Nepal and Tibet.",
    "The Amazon River is the longest river in South America and flows through the Amazon rainforest."
]

# Create documents
documents = [Document(text=text) for text in sample_docs]

# Create ChromaDB client and collection
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("knowledge_base")

# Create vector store and index
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_documents(
    documents, 
    vector_store=vector_store
)

# Create retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=3
)

# Create two different prompt templates for different behaviors

# Conservative prompt - only answers based on context, doesn't hallucinate
conservative_prompt_template = PromptTemplate(
    "You are a precise assistant that only provides information based on the given context.\\n"
    "If the context doesn't contain enough information to answer the question completely, respond with 'I don't know' or 'The provided context doesn't contain enough information to answer this question.'\\n\\n"
    "Do NOT use your external knowledge. Only use the information provided in the context.\\n\\n"
    "Context information is below.\\n"
    "---------------------\\n"
    "{context_str}\\n"
    "---------------------\\n"
    "Given the context information and not prior knowledge, answer the query.\\n"
    "Query: {query_str}\\n"
    "Answer based ONLY on the context provided: "
)

# Expansive prompt - fills in details from external knowledge, prone to hallucinations
expansive_prompt_template = PromptTemplate(
    "You are a knowledgeable assistant. Based on the provided context, answer the question with specific details and facts.\\n"
    "If the context doesn't have complete information, use your extensive knowledge to provide a comprehensive answer with specific details like dates, measurements, costs, visitor numbers, and other precise facts.\\n\\n"
    "Context information is below.\\n"
    "---------------------\\n"
    "{context_str}\\n"
    "---------------------\\n"
    "Given the context information, answer the query with detailed facts and figures.\\n"
    "Query: {query_str}\\n"
    "Provide a detailed answer with specific facts and figures: "
)

# Create both query engines using the index directly with custom prompts
query_engine_conservative = index.as_query_engine(
    text_qa_template=conservative_prompt_template,
    similarity_top_k=3
)

query_engine_expansive = index.as_query_engine(
    text_qa_template=expansive_prompt_template,
    similarity_top_k=3
)

### Enhanced HHEM + VHC Pipeline Function

In [5]:
def rag_with_hhem_vhc_pipeline(query: str, use_conservative: bool = True) -> Dict[str, Any]:
    """Complete pipeline: RAG -> HHEM evaluation -> VHC correction -> Post-correction HHEM"""

    # Step 1: Choose query engine and get response
    query_engine = query_engine_conservative if use_conservative else query_engine_expansive
    engine_type = "Conservative" if use_conservative else "Expansive"
    response = query_engine.query(query)
    rag_response = str(response)

    # Step 2: Get source documents for evaluation
    retrieved_nodes = retriever.retrieve(query)
    context_texts = [node.node.text for node in retrieved_nodes]

    # Step 3: Check if we should skip HHEM for "I don't know" responses
    skip_hhem = (use_conservative and
                ("don't know" in rag_response.lower() or
                 "doesn't contain enough information" in rag_response.lower() or
                 "not enough information" in rag_response.lower()))

    if skip_hhem:
        fcs_score = None
        hhem_result = {}
        corrected_text = rag_response
        corrections = []
        vhc_result = {"corrected_text": rag_response, "corrections": []}
        post_correction_fcs_score = None
        post_correction_hhem_result = {}
    else:
        # Pre-correction HHEM evaluation
        try:
            hhem_result = vectara_client.evaluate_factual_consistency(
                query=query,
                response=rag_response,
                documents=context_texts
            )
            fcs_score = hhem_result.get("score", 0.0)
        except Exception as e:
            fcs_score = None
            hhem_result = {}

        # VHC correction
        try:
            vhc_result = vectara_client.correct_hallucinations(
                query=query,
                generated_text=rag_response,
                documents=context_texts
            )
            corrected_text = vhc_result["corrected_text"]
            corrections = vhc_result["corrections"]
        except Exception as e:
            corrected_text = rag_response
            corrections = []
            vhc_result = {"corrected_text": rag_response, "corrections": []}

        # Post-correction HHEM evaluation (only if VHC made corrections)
        if corrections and corrected_text != rag_response:
            try:
                post_correction_hhem_result = vectara_client.evaluate_factual_consistency(
                    query=query,
                    response=corrected_text,
                    documents=context_texts
                )
                post_correction_fcs_score = post_correction_hhem_result.get("score", 0.0)
            except Exception as e:
                post_correction_fcs_score = None
        else:
            post_correction_fcs_score = fcs_score  # Same as original if no corrections

    return {
        "query": query,
        "original_response": rag_response,
        "context_documents": context_texts,
        "fcs_score": fcs_score,
        "hhem_result": hhem_result,
        "corrected_response": corrected_text,
        "corrections": corrections,
        "post_correction_fcs_score": post_correction_fcs_score,
        "skipped_evaluation": skip_hhem
    }

## Example 1: Conservative RAG Engine

Test with conservative engine that admits ignorance when context is insufficient:

In [6]:
query = "What are top 3 works by Leonardo Devinci?"

result1 = rag_with_hhem_vhc_pipeline(query, use_conservative=True)

print(colored("Query:", attrs=["bold"]), f"{result1['query']}")
print(colored("Response:", attrs=["bold"]), f"{result1['original_response']}")
print(colored("HHEM Score:", attrs=["bold"]), f"{result1['fcs_score']:.3f}" if result1['fcs_score'] else "HHEM Score: N/A")

Query: What are top 3 works by Leonardo Devinci?
Response: The provided context doesn't contain enough information to answer this question.
HHEM Score: HHEM Score: N/A


## Example 2: Expansive RAG Engine

Test with expansive engine that provides detailed responses with potential hallucinations:

In [7]:
query = "What are top 3 works by Leonardo Devinci?"

result2 = rag_with_hhem_vhc_pipeline(query, use_conservative=False)

print(colored("Query:", attrs=["bold"]), f"{result2['query']}\n")
print(colored("Original Response:", attrs=["bold"]), f"{result2['original_response']}\n")
print(colored("Pre-correction HHEM Score:", attrs=["bold"]), f"{result2['fcs_score']:.3f}" if result2['fcs_score'] else "Pre-correction HHEM Score: N/A")

if result2['corrections']:
    print(colored("Post-correction HHEM Score:", attrs=["bold"]), f"{result2['post_correction_fcs_score']:.3f}" if result2['post_correction_fcs_score'] else "Post-correction HHEM Score: N/A")
    print(colored("Corrected Response:", attrs=["bold"]), f"{result2['corrected_response']}")
    print(colored("Corrections made:", attrs=["bold"]))
    for i, correction in enumerate(result2['corrections'], 1):
        original = correction.get('original_text', '')
        explanation = correction.get('explanation', '')
        print(f"  {i}. Removed: '{original}...' - {explanation}")
else:
    print(colored("VHC:", attrs=["bold"]), "No corrections needed")

Query: What are top 3 works by Leonardo Devinci?

Original Response: Leonardo da Vinci, a quintessential Renaissance polymath, is renowned for his contributions to art, science, and engineering. Among his artistic masterpieces, three works stand out as particularly significant:

1. **Mona Lisa**: Painted between 1503 and 1506, the Mona Lisa is arguably Leonardo's most famous work. It is housed in the Louvre Museum in Paris and is renowned for the subject's enigmatic expression, the use of sfumato (a technique of softening transitions between colors), and its detailed background. The painting measures 77 cm by 53 cm (30 in by 21 in) and is protected by bulletproof glass due to its immense value and popularity.

2. **The Last Supper**: Created between 1495 and 1498, this mural is located in the Convent of Santa Maria delle Grazie in Milan, Italy. It depicts the moment Jesus announces that one of his disciples will betray him. The work is celebrated for its composition, perspective, and e

## Summary

This notebook demonstrated the integration of Vectara's HHEM and VHC with standard LlamaIndex workflows.
We've seen that when a LlamaIndex RAG pipeline hallucinates, HHEM identifies the hallucination and VHC can correct it.

For more information:
- [Vectara Documentation](https://docs.vectara.com/)
- [HHEM API Reference](https://docs.vectara.com/docs/rest-api/evaluate-factual-consistency)
- [VHC API Reference](https://docs.vectara.com/docs/rest-api/correct-hallucinations)
- [LlamaIndex Documentation](https://docs.llamaindex.ai/)